# Artificial Neural Network - Yapay Sinir Ağları (Çok Katmanlı Algılayıcı)
- İnsan beyninin bilgi işleme şeklini referans alan sınıflandırma ve regresyon problemleri için kullanılabilen kuvvetli makine öğrenmesi algoritmalarından birisidir.
- En küçük hata ile tahmin yapabilecek katsayılara(ağırlıklara) erişmektir.
- Amaç, gerçek değerler ile algoritmayla tahmin edilmiş değerler arasındaki farkları minimuma indirmeye çalışmaktır. 
- Görüntü işleme problemlerine uyarlanan yapay sinir ağlarının Derin Öğrenme olarak adlandırılır.
- Nümerik verilerle çalışırken Çok Katmanlı Algılayıcı ya da Yapay Sinir Ağları olarak adlandırılır.

**Gerekli Kütüphaneler** 

In [9]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, mean_squared_error, r2_score, roc_auc_score, roc_curve, classification_report
from sklearn.preprocessing import scale, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [10]:
# Uyarı Mesajları ile Karşılaşmamak için bu kütüphaneyi kullanıyoruz.
from warnings import filterwarnings
filterwarnings("ignore", category=DeprecationWarning)
filterwarnings("ignore", category=FutureWarning)

# Veri Seti Hikayesi ve Problemi: Şeker Hastalığı Tahmini

In [11]:
df = pd.read_csv("./diabetes.csv")
# bu csv dosyasının içerisinde eksik gözlemleri(NA) çıkardık.
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


# Artificial Neural Network  Model ve Tahmin İşlemleri

In [12]:
y= df["Outcome"]
X = df.drop(["Outcome"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.30,
                                                    random_state = 42)

In [73]:
## Değişkenlerin Dönüştürülmesi 
scaler = StandardScaler()

In [69]:
scaler.fit(X_train)
X_train = scaler.transform(X_train)

In [70]:
scaler.fit(X_test)
X_test = scaler.transform(X_test)

**Model**

- Yapay sinir ağlarında kullanılan **activation** fonksiyonu **sınıflandırma problemleri** için "**logistic**" fonksiyonu ile kullanılması gerekir. Eğer bir **regresyon problemi** kullanılıyorsa "**relu**" fonksiyonu kullanılması gerekir.

In [80]:
#mlpc = Multi Layer Perseptron Classification
mlpc_model =MLPClassifier(solver="lbfgs", activation="logistic").fit(X_train,y_train)

In [81]:
?mlpc_model

Type:        MLPClassifier
String form: MLPClassifier(activation='logistic', solver='lbfgs')
File:        c:\users\halil\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py
Docstring:  
Multi-layer Perceptron classifier.

This model optimizes the log-loss function using LBFGS or stochastic
gradient descent.

.. versionadded:: 0.18

Parameters
----------
hidden_layer_sizes : tuple, length = n_layers - 2, default=(100,)
    The ith element represents the number of neurons in the ith
    hidden layer.

activation : {'identity', 'logistic', 'tanh', 'relu'}, default='relu'
    Activation function for the hidden layer.

    - 'identity', no-op activation, useful to implement linear bottleneck,
      returns f(x) = x

    - 'logistic', the logistic sigmoid function,
      returns f(x) = 1 / (1 + exp(-x)).

    - 'tanh', the hyperbolic tan function,
      returns f(x) = tanh(x).

    - 'relu', the rectified linear unit function,
      returns f(x) = max(0, x)

solver :

**Tahmin**

In [82]:
y_pred = mlpc_model.predict(X_test)
accuracy_score(y_test,y_pred)

0.6623376623376623

# Model Tuning (Model Doğrulama)
-  Modelin başarısını değerlendirmek için izlenecek yol Doğruluk oranına (accuracy_score) bakılarak yapılacak. 
- **accuracy_score**' u ne kadar büyük olursa sınıflandırma da o kadar başarılı olduğu anlamına gelir.
- Hiper parametrelerin optimum değerlerine ulaşmak içinde Cross-Validation yöntemini kullanmış olacağız.
- Yani modelleri Train Setiyle eğitirken Cross-Validation yönetemini kullanıp, dışarı bıraktığımız Test Setini ise performansımızı test etmek için kullanacaz.

In [30]:
mlpc_params = {"alpha":[1,5,0.1,0.01,0.03,0.005,0.0001],
               "hidden_layer_sizes":[(10,10), (100,100,100),(100,100),(3,5)]}

In [71]:
mlpc_model = MLPClassifier(solver="lbfgs", activation="logistic")

In [72]:
mlpc_cv_model = GridSearchCV(mlpc_model,
                            mlpc_params,
                            cv=10,
                            n_jobs= -1,
                            verbose= 2).fit(X_train,y_train)

Fitting 10 folds for each of 28 candidates, totalling 280 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 178 tasks      | elapsed:   31.6s
[Parallel(n_jobs=-1)]: Done 280 out of 280 | elapsed:   54.1s finished


In [74]:
mlpc_cv_model

GridSearchCV(cv=10,
             estimator=MLPClassifier(activation='logistic', solver='lbfgs'),
             n_jobs=-1,
             param_grid={'alpha': [1, 5, 0.1, 0.01, 0.03, 0.005, 0.0001],
                         'hidden_layer_sizes': [(10, 10), (100, 100, 100),
                                                (100, 100), (3, 5)]},
             verbose=2)

In [75]:
mlpc_cv_model.best_params_

{'alpha': 1, 'hidden_layer_sizes': (3, 5)}

### Final Modeli

In [76]:
mlpc_tuned = MLPClassifier(activation='logistic',solver="lbfgs",alpha=1, hidden_layer_sizes = (3,5)).fit(X_train,y_train)

In [77]:
y_tuned_pred = mlpc_tuned.predict(X_test)
accuracy_score(y_test,y_tuned_pred)

0.7575757575757576